In [ ]:
import numpy as np
import os
import scipy.linalg as linalg
from scipy.stats import multivariate_normal
import cv2
from datetime import datetime, timedelta
import time

In [ ]:
# take a series of images within video
def test_image(test_path, resolution):
    """
    Take 5 test images with face from vedio
    
    Input:
    -image_path: path of image folder to store captured images.
    -resolution: resize resolution, (px, px)
    
    Output:
    -n images with faces stored in designiated image folder.
    """
    # senitize path and create folder if not exists
    test_path = test_path
    if not test_path.endswith('\\'):
        test_path +='\\'
    if not os.path.exists(test_path):
        os.mkdir(test_path)

    img_id = 0
    start = False
    
    oldtime = datetime.now()
    
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    flatimages = list()
    
    while(img_id<5):
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Our operations on the frame come here
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # face tracking on gray image
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Display the resulting frame  
        for (x,y,w,h) in faces:
            cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
        
        # Keyboard interpution or start image acquire.
        if cv2.waitKey(1) & 0xFF == ord('s'):
            start = True
            oldtime = datetime.now()
        elif cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        # take image after 1 second if face is detected.
        newtime = datetime.now()
        diff = (newtime-oldtime).seconds

        # take 5 face samples as query
        if len(faces) and start and diff>1:               
            filename = 'img_%i.jpg'%img_id
            img_gray = cv2.resize(roi_gray, resolution)
            cv2.imwrite(str(test_path)+filename,img_gray)
            flatimages.append(img_gray.ravel())
            img_id+=1
            oldtime = datetime.now()
            
        # instruction notes show in the video
        if not start:
            cv2.putText(gray, "Press s to start, press q to quit!", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,255,255),2)
        else:
            cv2.putText(gray, "Analyzing {0}%".format(img_id*20), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,255,255),2)
        
        cv2.imshow('frame',gray)
        faces = None

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    
    return np.asarray(flatimages)

In [ ]:
query = test_image('test', (50,50))

In [ ]:
query

In [ ]:
npzfile = np.load('mean_eigenvectors_sigma.npz')
mu = npzfile['mu']
V = npzfile['V'][:, 0:2]


In [ ]:
npzfile = np.load('class0_stats.npz')
N0 = npzfile['N0'][0]
mu0 = npzfile['mu0']
c0 = npzfile['c0']

In [ ]:
npzfile = np.load('class1_stats.npz')
N1 = npzfile['N1'][0]
mu1 = npzfile['mu1']
c1 = npzfile['c1']

In [ ]:
def pred_query(query):
    P2 = np.dot(query-mu, V)
    r
    pdf0 = multivariate_normal.pdf(P2, mean=mu0, cov=c0)
    pdf1 = multivariate_normal.pdf(P2, mean=mu1, cov=c1)
    probs_0 = N0*pdf0/(N0*pdf0+N1*pdf1)
    probs_1 = N1*pdf1/(N0*pdf0+N1*pdf1)

    pred = np.zeros((5,))
    pred.fill(np.nan)
    pred[probs_0>probs_1] = 0
    pred[probs_0<probs_1] = 1
    
    return pred

In [13]:
P2 = np.dot(query-mu, V)
pdf0 = multivariate_normal.pdf(P2, mean=mu0, cov=c0)r
pdf1 = multivariate_normal.pdf(P2, mean=mu1, cov=c1)

In [ ]:
pdf0, pdf1

In [ ]:
probs_0 = N0*pdf0/(N0*pdf0+N1*pdf1)
probs_1 = N1*pdf1/(N0*pdf0+N1*pdf1)

pred = np.zeros((5,))
pred.fill(np.nan)
pred[probs_0>probs_1] = 0
pred[probs_0<probs_1] = 1

In [ ]:
pdf0 = multivariate_normal.pdf(P2, mean=mu0, cov=c0)
pdf1 = multivariate_normal.pdf(P2, mean=mu1, cov=rc1)
probs_0 = N0*pdf0/(N0*pdf0+N1*pdf1)
probs_1 = N1*pdf1/(N0*pdf0+N1*pdf1)

pred = np.zeros_like((5,))
pred.fill(np.nan)
pred[probs_0>probs_1] = 0
pred[probs_0<probs_1] = 1

In [ ]:
pred = pred_query(query)    

In [ ]:
predicted_classes = np.unique(pred) 
bin_counts = list()
for i in range(len(predicted_classes)):
    bin_counts.append(np.sum(pred==predicted_classes[i]))

counts = bin_counts[np.argmax(bin_counts)]
class_id = predicted_classes[np.argmax(bin_counts)]r
class_id, counts

if class_id == 0:
    print ("Predict Fei")
elif class_id == 1:
    print ("Predict Weiya")
elif class_id == np.nan:
    print ("Can not predict")
